# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore.
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules:

In [1]:
import numpy as np

import mbuild as mb
from grits import CG_System

from morphct.system import System
from morphct.chromophores import amber_dict

/Users/jenny/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


In [55]:
#itic_backbone = "c1c4c(cc2c1Cc3c2sc(C=C1C(=C(C#N)C#N)C2=CC=CC=C2C1(=O))c3)Cc5c4sc(C=C1C(=C(C#N)C#N)C2=CC=CC=C2C1(=O))c5"
itic_backbone = "c1c4c(cc2c1Cc3c2scc3)Cc5c4scc5"
itic_end = "C=C1C(=C(C#N)C#N)C2=CC=CC=C2C1(=O)"
backbone = mb.load(itic_backbone, smiles=True)
backbone.visualize().show()
end = mb.load(itic_end, smiles=True)
end.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In the cell below, we'll create a system object for morphct. This the main class that will hold all the information for our simulation. We'll need to give it a gsd file, path to an output directory, the frame of the gsd file to use, the scaling factor to convert the lengths in the gsd to Angstroms, and a dictionary to map particle types to elements. Here's our starting structure, an atomistic (not coarse-grain or united atom) gsd file with 10 ITIC molecules:

In [12]:
gsdfile = "itic_trajectory.gsd"

system = System(gsdfile, "output_itic", frame=-1, scale=3.5636, conversion_dict=amber_dict)
system.visualize_system()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next we need to pick the chromophores. We'll use GRiTS, as in the P3HT workflow.

In [54]:
cg_system = CG_System(gsdfile, beads={"_B": itic_backbone}, conversion_dict=amber_dict)

/Users/jenny/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/grits/coarsegrain.py:139: UserWarning: C1C(=C(C#N)C#N)C2=CC=CC=C2C1(=O) not found in compound!
  warn(f"{smart_str} not found in compound!")


Next let's add these chromophores to the system and visualize them. The acceptors will be colored dark red and the donors will be colored purple.

The reorganization energy for ITIC comes from [Han 2018](https://doi.org/10.1002/solr.201800251).

In [59]:
system.add_chromophores(
    cg_system.mapping['_B...c1c4c(cc2c1Cc3c2scc3)Cc5c4scc5'], 
    "acceptor", 
    chromophore_kwargs={"reorganization_energy": 0.15}
)

system.visualize_chromophores()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next let's compute the energies required to run the KMC simulation. First, the neighbors will be calculated (using voronoi analysis) and then the single and dimer energies will be calculated and saved to a file. (So that the simulation can be restarted from this point more easily.)

In [ ]:
system.compute_energies()

We can check that the pair and singles inputs look reasonable. There won't be any bonds and hydrogen atoms should've been added.

In [ ]:
i = 86 # try any number from 0 to 86
print(f"Pair #{i}:")
system.visualize_qcc_input(i, single=False)

i = 0 # try any number from 0 to 9
print(f"Single #{i}:")
system.visualize_qcc_input(i)

Once the energy files are finished, we can use them to set the energy values of the chromophores . (This can also be run to restart the calculation from this point.)

In [ ]:
system.set_energies()

This function sets the `homo_1`, `homo`, `lumo`, `lumo_1`, `neighbors_delta_e`, and `neighbors_ti` attribute of each chromphore:

In [ ]:
i = 0
chromo = system.chromophores[i]
print(f"Chromophore {i}:")
print(
    f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} "
    f"LUMO+1: {chromo.lumo_1:.2f}"
)
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

With all the energy values set, we're ready to run KMC! We need to set the temperature that the KMC simulation will be run at and the lifetimes and numbers of our carriers:

In [ ]:
lifetimes = [1.00e-13, 1.00e-12]
temp = 300
system.run_kmc(lifetimes, temp, n_holes=10, n_elec=10, verbose=1)

The output files for each process are saved in `output_itic/kmc/kmc_PROC#.log` (where `PROC#` is whatever process number the job was run on) and analysis plots will be saved in `output_itic/kmc/figures/`.

In [ ]:
with open("output_itic/kmc/kmc_00.log", "r") as f:
    lines = f.readlines()
print(*lines)